# Tutorial on linear dynamical systems

In this notebook, we'll go over examples of different linear dynamical systems.

In [ ]:
#!/usr/bin/python

import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import ssm
from ssm import util
import LDS
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
# enable relative imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import plotting_util
import tutorial_util

## Generate synthetic data from a linear dynamical system
First, we generate states and observations from a linear dynamical system with gaussian noise. The dynamical system has the form:

\begin{equation}
\vec{x}_{t+1} = A\vec{x}_t + \vec{\omega}, \text{where}~\vec{\omega} \sim N(0,Q)
\end{equation}

With observations:

\begin{equation}
\vec{y}_{t+1} = C\vec{x}_{t+1} + \vec{\eta}, \text{where}~\vec{\eta} \sim N(0,R)
\end{equation}

The initial state $\vec{x}_0$ also comes from a gaussian distribution $N(\pi_0,V_0)$. 

Fill in the function `glds_generate_states_and_observations` below. **Fill in the code** to generate new latent states and observations. A few useful numpy functions:

- [np.matmul(A,b)](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.dot.html): matrix multiplication between A and b, i.e. $A\vec{b}$
- [np.random.multivariate_normal($\mu$, $\Sigma$)](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.dot.html): generate a sample $\vec{\omega} \sim N(\mu,\Sigma)$

In [ ]:
def glds_generate_states_and_observations(A, C, Q, R, pi_0, V_0, num_timesteps, seed=None):
    """ Generates a sequence of states and observations from a gaussian linear dynamical system.

    Args:
        A (np.matrix): Dynamics matrix (d_latent, d_latent)
        Q (np.matrix): State noise covariance (d_latent, d_latent)
        C (np.matrix): Observation matrix (d_observation, d_latent)
        R (np.matrix): Observation noise covariance (d_observation, d_observation)
        pi_0 (np.array): Initial state mean (d_latent, )
        V_0 (np.matrix): Initial state covariance (d_latent, d_latent)
        num_timesteps (int, optional): number of iterations for EM

    Returns:
        X (np.ndarray): (num_timesteps, d_latent) time-series of states
        Y (np.ndarray): (num_timesteps, d_observations) time-series of observations
    """
    if seed is not None:
        np.random.seed(seed) # for testing; do not change
    assert A.shape[0] == A.shape[1], "Dynamics matrix must be square"
    assert Q.shape[0] == Q.shape[1], "State noise covariance must be square"
    assert C.shape[1] == A.shape[1], "Number of columns in observation matrix must match d_latent "
    assert R.shape[0] == R.shape[1], "Observation noise covariance must be square"
    
    d_latent = A.shape[0]
    d_observation = C.shape[0]

    X = [] # list of states 
    Y = [] # list of observations
    
    # use these!
    state_noise_mean = np.zeros((d_latent,))
    observation_noise_mean = np.zeros((d_observation,))
    
    # generate initial state and observation
    x = np.random.multivariate_normal(pi_0, V_0)
    y = C.dot(x) + np.random.multivariate_normal(observation_noise_mean, R)
    
    # add x and y to their respective lists
    X.append(x)
    Y.append(y)
    

    
    for _ in range(1, num_timesteps):
        """TODO: your code goes here! Fill in the formulas for x and y.
        
        Note: just as we did in the initialization, we're actually finding x_{t+1} and y_{t+1} at each timestep.
        This means that your equation for y should be in terms of the _new_ x, not the previous x.
        """
    
        x = 0
        y = 0
        
        """End your code"""
        X.append(x)
        Y.append(y)
    
    X = np.array(X)
    Y = np.array(Y)
    
    return X, Y

In [ ]:
# run this to test your implementation
tutorial_util.test_implementation(glds_generate_states_and_observations)

# Visualizing linear dynamical systems

Now we'll see a few different ways of visualizing linear dynamical systems. Let's initialize some parameters:

In [ ]:
# parameters of the gLDS
d_latent = 2
d_observation = 3
A = .95* util.random_rotation(d_latent,theta=-np.pi/6) # dynamics matrix, a slowly decaying rotation
C = np.random.rand(d_observation,d_latent) # observation matrix, random

# we'll set the covariances to be diagonal
Q = np.diag(np.random.rand(d_latent,)) # state noise covariance
R = np.diag(np.random.rand(d_observation,)) # observation noise covariance

pi_0 = np.zeros((d_latent,)) # initial state mean
V_0 = Q # initial state covariance

In [ ]:
plotting_util.plot_vector_field(A)

Using these parameters and the function we wrote above, let's generate 200 timesteps of states $x_t$ and observations $y_t$.

In [ ]:
num_timesteps = 200
X, Y = glds_generate_states_and_observations(A, C, Q, R, pi_0, V_0, num_timesteps)

First, let's plot each dimension of the latent state separately over time.

In [ ]:
plotting_util.plot_latents_over_time(X)

We can do the same thing for the observations:

In [ ]:
plotting_util.plot_observations_over_time(Y)

Another way to visualize the latents is by looking at the phase space -- now, our axes are the values of the latent dimensions, and our plot evolves over time. We've linked the phase space to the observations so we can see how the two relate to one another:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
anim = plotting_util.scatter_animation_2D_and_3D(X, Y)
HTML(anim.to_html5_video())

# Flow fields

We can summarize the dynamics with a flow field. The arrows show how, in the absence of noise, the latent state evolves over time. For example, here's the flow field when our dynamics matrix $A$ is a rotation matrix:

In [ ]:
theta = np.pi/6 # angle of rotation in radians
rotation_matrix = np.matrix([[np.cos(theta), np.sin(theta)],
                   [-np.sin(theta),np.cos(theta)]])
plotting_util.plot_vector_field(rotation_matrix)

Here are flow fields when our dynamics matrices collapse the state or expand the state:

In [ ]:
collapsing_dynamics = np.matrix([[.9, 0],
                                [0,.9]])
expanding_dynamics = np.matrix([[1.1, 0],
                               [0,1.1]])
plotting_util.plot_vector_field(collapsing_dynamics, expanding_dynamics)

As a side note, we can figure out which of these qualitative behaviors will occur based on the eigenvalues of the dynamics matrices,

In [ ]:
rotation_eigenvalues = np.linalg.eig(rotation_matrix)[0]
print("Eigenvalues of rotation matrix: {}".format(rotation_eigenvalues))

collapse_eigenvalues = np.linalg.eig(collapsing_dynamics)[0]
print("Eigenvalues of collapsing matrix: {}".format(collapse_eigenvalues))

expanding_eigenvalues = np.linalg.eig(expanding_dynamics)[0]
print("Eigenvalues of expanding matrix: {}".format(expanding_eigenvalues))

We can combine these qualitative behaviors to get rotating + collapsing dynamics, rotating + expanding dynamics, or collapsing + expanding (along different dimensions):

In [ ]:
rotating_collapse = collapsing_dynamics.dot(rotation_matrix)
rotating_expand = expanding_dynamics.dot(rotation_matrix)
plotting_util.plot_vector_field(rotating_collapse, rotating_expand)

## Challenge
Fill in the entries of this matrix to generate a matrix that collapses along one dimension and expands along the other

In [ ]:
# uncomment these lines
# collapsing_expand = np.matrix([[?,?],
#                                [?,?]])
# plotting_util.plot_vector_field(collapsing_expand)

# Fitting Linear Dynamical Systems

Now we'll see a simple example of fitting a linear dynamical system. We'll start with a toy example using the observations that we generated above, and learn the parameters just from these observations.

The actual fitting work is happening in the [LDS.py](LDS.py) module. When we want to fit a dynamical system to a set of observations, we call the module's fit() function, which looks like this:

```python
for _ in range(num_iterations):
    A, C, Q, R, pi_0, V_0 = self._m_step(x_s, V_s, ...)
    x_s, V_s, ... = self._e_step(y, A, C, Q, R, pi_0, V_0)

```

The fit function implements EM, which alternates between 2 steps:
    - E step (given current estimate of parameters, find most likely latent states)
    - M step (given current estimate of latent states, find most likely parameters)
    
    
Note: [LDS.py](LDS.py) is mostly for educational purposes only; it implements [Parameter estimation for linear dynamical systems](http://mlg.eng.cam.ac.uk/zoubin/course04/tr-96-2.pdf), but is generally only suitable for toy examples. [SSM](https://github.com/slinderman/ssm), a much more robust library, is presented later in the tutorial.

In [ ]:
model = LDS.gLDS() # instantiate gaussian linear dynamical systems module
x_hat, model_likelihood = model.fit(Y, d_latent, num_iterations=100)

In [ ]:
true_likelihood = LDS.gLDS.get_likelihood(Y, A, C, Q, R, pi_0, V_0)
fig, ax = plt.subplots(1,1)
ax.hlines(true_likelihood, 0, len(model_likelihood), label='true')
ax.plot(model_likelihood, label='model')
ax.set_xlabel('Iterations')
ax.set_ylabel('Log likelihood')
ax.legend()

Note that the model likelihood can pass the "true" likelihood, because there could be other parameters that were more likely to have produced the observations.

We can compare the dynamics matrix found by the model to the true dynamics matrix by looking at the flow fields:

In [ ]:
plotting_util.plot_vector_field(model.A, A)

Note that the estimated dynamics may be the mirror image of the 'true' dynamics because both can give rise to the same set of observations as if the columns of the model's C have their signs flipped relative to the 'true' C. What matters is that the eigenvalues are similar:

In [ ]:
import numpy.linalg as LA
print(LA.eigvals(A))
print(LA.eigvals(model.A))

We can also compare the estimated latent states vs. the true latent states. Here are the true states:

In [ ]:
plotting_util.plot_latents_over_time(X)

and here are the estimates:

In [ ]:
plotting_util.plot_latents_over_time(x_hat)

So hopefully you're now convinced that it's possible to fit a linear dynamical system from data. Now that we've gone over a simple case, we'll dive into an example with real data.

# Fitting dynamical systems to C. elegans postures

Now that we're familiar with the basics of fitting and visualizing dynamical systems, we'll go over an example using real data. We'll be working with a dataset of C. elegans behavior from Andre Brown's [Behavioural Genomics Lab](http://behave.lms.mrc.ac.uk/). The videos have been preprocessed using the [Tierpsy Tracker](http://ver228.github.io/tierpsy-tracker/). 

In [ ]:
import worm_util
worm_data = worm_util.load_worm_data()
for key in worm_data.keys():
    print(key, "shape: {}".format(worm_data[key].shape))

The `coordinates` field shows the inferred coordinates of the worm as it crawls around the plate. The worm is segmeneted into 48 different sections, and we have the x, y coordinates of each of the segment endpoints. Here's an animation of the worm's extracted position and body as it moves on the plate (note -- a camera is tracking the worm, hence the jumps):

In [ ]:
num_frames = 300
anim = plotting_util.plot_crawling_worm(worm_data['coordinates'][:num_frames], worm_data['fps'])
HTML(anim.to_jshtml())

We are going to build a dynamical systems model of how the worm's `posture` changes over time. To get the posture, we center the worm and set the mean angle of it's body equal to 0:

In [ ]:
anim = plotting_util.plot_crawling_worm(worm_data['posture'][:num_frames], worm_data['fps'])
HTML(anim.to_jshtml())

## Eigen worms

Before fitting the dynamical systems model, we're going to reduce the dimensionality of our dataset from 48 joint segments to 7 using PCA. [Stephens et. al 2008](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1000028) showed that 95% of the variance in C. elegans posture is explained by the top 4 principal components, or 'eigenworms'. Also, following Stephens et. al, the eigen worms are represented in terms of the tangent angles along the segments of the worm's body. We've kept the top 7 PCs which explain 98% of the variance in the postures. Here's what the eigenworms look like:

In [ ]:
plotting_util.plot_eigen_worms(worm_data['eigen_worms'])

The posture at any given time will be a linear combination of the seven eigenworms. Here are all of the `eigen_projections` for the full movie:

In [ ]:
plotting_util.plot_eigen_projections(worm_data['eigen_projections'], worm_data['fps'])

Let's look at the first two trajectories plotted against one another:

In [ ]:
anim = plotting_util.scatter_animation_2D(worm_data['eigen_projections'][:90,1:3])
HTML(anim.to_jshtml())

Now let's try to model these dynamics!

## State-space modeling library

Luckily, there are several open source libraries available for fitting state space models. We're going to use [SSM: Bayesian learning and inference for state space models](https://github.com/slinderman/ssm), a library by Scott Linderman (joining SNI and the Statistics department June 2019!). SSM provides a simple interface for fitting a number of different state space models.



In [ ]:
from ssm.models import HMM
from ssm.util import find_permutation

## Switching linear dynamical system

*Note: This model is directly taken from "Quantifying the behavioral dynamics of C. elegans with autoregressive hidden Markov models", a 2017 NeurIPS workshop paper by Buchanan, E. K., Lipschitz, A., Linderman, S. W., & Paninski, L.*

We're going to fit a specific type of linear dynamical system called an auto-regressive hidden markov model (AR-HMM). The AR-HMM differs from the LDS model above in two crucial ways:

1. Instead of having one dynamics matrix $A$, we'll $K$ different dynamics matrices $A_1, \ldots, A_K$. We can think of the $K$ different sets of dynamics corresponding to $K$ different behaviors, each of which we model as being linear.

2. For simplicity, we're going to drop the observations for now, and only try to model the dynamics of the eigen projections. We could have instead fit a full switching linear dynamical system where the observations were the original 48-dimensional vector...but that takes more time to fit. Try it on your own!

The equations for the AR-HMM are as follows: we have the discrete states

$$z _ { t + 1 } \left| z _ { t } , \left\{ \pi _ { k } \right\} _ { k = 1 } ^ { K } \sim \pi _ { z _ { t } }\right.$$

where is $\left\{ \pi _ { k } \right\} _ { k = 1 } ^ { K }$ the Markov transition matrix and $\pi _ { k } \in [ 0,1 ] ^ { K }$ corresponds to the $k$-th row. Given discrete state $z_t$, the postural dynamics are given by

$$x _ { t } \left| x _ { t - 1 } , z _ { t } \sim \mathcal { N } \left( A _ { z _ { t } } x _ { t - 1 } + b _ { z _ { t } } , Q _ { z _ { t } } \right)\right.$$

In [ ]:
# Set the parameters of the HMM
T = worm_data['eigen_projections'].shape[0]    # number of time bins
K = 4       # number of discrete states
D = worm_data['eigen_projections'].shape[1]       # data dimension

# Make an HMM, use direchlet prior on discrete states so high prob of staying in current state
hmm = HMM(K, D, observations="ar", transitions='sticky', transition_kwargs={'kappa': np.power(10,4)})
hmm.initialize(worm_data['eigen_projections'])
hmm_lls = hmm.fit(worm_data['eigen_projections'], method="em", num_em_iters=20)

plt.plot(hmm_lls, label="EM")
plt.xlabel("EM Iteration")
plt.ylabel("Log Probability")
plt.legend(loc="lower right")

In [ ]:
# Plot the inferred discrete states
hmm_z = hmm.most_likely_states(worm_data['eigen_projections'])
plotting_util.plot_discrete_states(hmm_z, worm_data['max_time_in_seconds'])

Let's see if the AR-HMM learned any interested structure in our data. First, we can show a histogram of how the different discrete states correspond to hand-labeled behavioral states.

In [ ]:
plotting_util.plot_discrete_state_behavioral_histograms(hmm_z, worm_data['mode'])

It looks like one discrete state roughly corresponds to backwards crawling, the ~2 are forwards crawling, and the last might be pausing. 

We can also visualze the flowfields of the corresponding dynamics. Because the dynamics are in a 7-dimensional space, we visualize the dynamics projected onto the 2D subspace spanned by PCs 2 and 3 (eigen_worms):

In [ ]:
xkcd = np.array(plotting_util.xkcd_colors())
for z in range(K):
    A = hmm.observations.params[0][z,1:3,1:3]
    plotting_util.plot_vector_field(A, color=xkcd[z])

Lastly, now that we have a generative model of the worms behavior, we can sample from the hmm to simulate new worm experiments!

In [ ]:
sampled_z, sampled_eigen_projections = hmm.sample(600)
sampled_worm_postures = worm_util.angles_to_skeleton(worm_data['eigen_worms'].dot(sampled_eigen_projections.T)).transpose(1,0,2)
# color worm by discrete state

z = xkcd[sampled_z,:]
anim = plotting_util.plot_crawling_worm(sampled_worm_postures, worm_data['fps'],colors=z)
HTML(anim.to_html5_video())